# Raster Processing with GeoProcessor

This notebook demonstrates how to use the GeoProcessor library for common raster data processing tasks. We'll use Rasterio, GDAL, NumPy, and xarray to perform operations like:

- Loading and visualizing raster data
- Reprojecting rasters to different coordinate systems
- Clipping rasters to vector geometries
- Calculating vegetation indices (NDVI)
- Converting between raster formats and xarray datasets
- Performing raster calculations

Let's get started!

## Setup and Installation

First, let's make sure we have all the necessary packages installed:

In [ ]:
# Install required packages if not already available
!pip install rasterio xarray numpy pandas matplotlib geopandas boto3 shapely

In [ ]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
import rasterio
from rasterio.plot import show
from rasterio.warp import calculate_default_transform, reproject
import matplotlib.pyplot as plt
import geopandas as gpd
import xarray as xr
from shapely.geometry import box, mapping

# For downloading sample data
import urllib.request
import zipfile
import tempfile

# Set up display parameters
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

## Downloading Sample Data

For this tutorial, we'll use Landsat 8 sample data. Let's download a sample scene:

In [ ]:
def download_landsat_sample():
    """Download a sample Landsat 8 scene"""
    # Create data directory if it doesn't exist
    data_dir = os.path.join(os.getcwd(), 'data', 'raster')
    os.makedirs(data_dir, exist_ok=True)
    
    # Check if data already exists
    landsat_dir = os.path.join(data_dir, 'landsat8_sample')
    if os.path.exists(landsat_dir):
        print(f"Sample data already exists at {landsat_dir}")
        return landsat_dir
    
    # URL for Landsat 8 sample data (small subset)
    # This is a URL to an Earth Explorer sample - you can replace with your own data source
    url = "https://github.com/sentinelsat/sentinelsat/raw/master/tests/data/S2A_MSIL1C_20170102T111442_N0204_R137_T30TXT_20170102T111441.zip"
    
    print("Downloading sample data...")
    with tempfile.NamedTemporaryFile(suffix='.zip', delete=False) as tmp_file:
        urllib.request.urlretrieve(url, tmp_file.name)
        
        # Extract the zip file
        os.makedirs(landsat_dir, exist_ok=True)
        with zipfile.ZipFile(tmp_file.name, 'r') as zip_ref:
            zip_ref.extractall(landsat_dir)
    
    print(f"Sample data downloaded to {landsat_dir}")
    return landsat_dir

# Download the sample data
landsat_dir = download_landsat_sample()

## Alternative: Using Google Colab to Access Rasterio Sample Data

If you're running this notebook in Google Colab, you can use the following code to access sample data from Rasterio's built-in datasets:

In [ ]:
# This will only work in Google Colab
# First, install Rasterio if not already installed
!pip install rasterio

# Get sample data path from Rasterio's testing resources
import rasterio
from rasterio.plot import show

# Check if we're in Colab
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    # Get Rasterio's sample data
    !pip install pytest
    import pytest
    !pytest -xvs --collect-only rasterio
    
    # Download a sample GeoTIFF
    !wget https://github.com/mapbox/rasterio/raw/master/tests/data/RGB.byte.tif -O sample.tif
    
    sample_raster_path = 'sample.tif'
    print(f"Sample raster downloaded to {sample_raster_path}")
else:
    # If not in Colab, try to use the previously downloaded data
    sample_files = os.listdir(landsat_dir)
    tif_files = [f for f in sample_files if f.endswith('.tif')]
    if tif_files:
        sample_raster_path = os.path.join(landsat_dir, tif_files[0])
    else:
        sample_raster_path = None
        print("No sample raster files found.")

## 1. Loading and Visualizing Raster Data

Let's start by loading and visualizing a raster dataset:

In [ ]:
def visualize_raster(raster_path):
    """Visualize a raster dataset with its metadata"""
    with rasterio.open(raster_path) as src:
        # Read the data
        img = src.read()
        
        # Print metadata
        print(f"Dimensions: {src.width} x {src.height}")
        print(f"Number of bands: {src.count}")
        print(f"Coordinate reference system: {src.crs}")
        print(f"Bounds: {src.bounds}")
        print(f"Transform: {src.transform}")
        print(f"Data type: {src.dtypes[0]}")
        
        # Plot the raster
        fig, ax = plt.subplots(1, figsize=(12, 8))
        
        # Handle different band counts
        if src.count == 1:
            show(src, ax=ax, cmap='viridis')
        elif src.count >= 3:
            # Create an RGB composite
            show(src.read([1, 2, 3]), transform=src.transform, ax=ax)
        else:
            # Multiple bands but not 3
            show(src.read(1), transform=src.transform, ax=ax, cmap='viridis')
            
        plt.title("Raster Visualization")
        plt.tight_layout()
        plt.show()

# Visualize the sample raster
visualize_raster(sample_raster_path)

## 2. Reprojecting Rasters

Now, let's see how to reproject a raster to a different coordinate system:

In [ ]:
# Import the function from our package
from geoprocessor.raster.processing import reproject_raster

# Reproject to WGS84 (EPSG:4326)
output_path = 'reprojected_sample.tif'
reproject_raster(sample_raster_path, output_path, 'EPSG:4326')

# Visualize the reprojected raster
print("Original raster:")
visualize_raster(sample_raster_path)
print("\nReprojected raster:")
visualize_raster(output_path)

## 3. Clipping Rasters to Geometries

Let's clip our raster to a specific geographic extent:

In [ ]:
# Import the function from our package
from geoprocessor.raster.processing import clip_raster_to_geometry

# First, let's determine the bounds of our sample raster
with rasterio.open(sample_raster_path) as src:
    raster_bounds = src.bounds
    
# Create a smaller geometry for clipping (using only the central portion)
minx = raster_bounds.left + (raster_bounds.right - raster_bounds.left) * 0.25
miny = raster_bounds.bottom + (raster_bounds.top - raster_bounds.bottom) * 0.25
maxx = raster_bounds.right - (raster_bounds.right - raster_bounds.left) * 0.25
maxy = raster_bounds.top - (raster_bounds.top - raster_bounds.bottom) * 0.25

# Create a box geometry
clip_geometry = box(minx, miny, maxx, maxy)
clip_geom_dict = mapping(clip_geometry)

# Clip the raster
clipped_path = 'clipped_sample.tif'
clip_raster_to_geometry(sample_raster_path, clip_geom_dict, clipped_path)

# Visualize the clipped raster
print("Original raster:")
visualize_raster(sample_raster_path)
print("\nClipped raster:")
visualize_raster(clipped_path)

## 4. Calculating Vegetation Indices (NDVI)

The Normalized Difference Vegetation Index (NDVI) is commonly used in remote sensing to analyze vegetation health. Let's calculate NDVI from a multi-band satellite image:

In [ ]:
# Import the function from our package
from geoprocessor.raster.processing import calculate_ndvi

# For demonstration purposes, we'll create synthetic red and NIR bands
# In a real application, you would have separate files for these bands

with rasterio.open(sample_raster_path) as src:
    # Create synthetic red and NIR bands from the sample data
    profile = src.profile.copy()
    data = src.read(1)  # Read the first band
    
    # Create synthetic bands - this is for demonstration only
    # In real application, use actual red (Band 4) and NIR (Band 5) from Landsat 8
    red_data = data.astype(np.float32) / 10000  # Simulated reflectance values
    nir_data = data.astype(np.float32) / 8000  # Simulated reflectance values
    
    # Save to temporary files
    red_path = 'temp_red.tif'
    nir_path = 'temp_nir.tif'
    
    profile.update(dtype=rasterio.float32, count=1)
    with rasterio.open(red_path, 'w', **profile) as dst:
        dst.write(red_data, 1)
    with rasterio.open(nir_path, 'w', **profile) as dst:
        dst.write(nir_data, 1)

# Calculate NDVI
ndvi_path = 'ndvi_result.tif'
ndvi = calculate_ndvi(red_path, nir_path, ndvi_path)

# Visualize the NDVI
plt.figure(figsize=(12, 8))
with rasterio.open(ndvi_path) as src:
    ndvi_data = src.read(1)
    show(ndvi_data, cmap='RdYlGn', vmin=-1, vmax=1, title='NDVI')
plt.colorbar(label='NDVI')
plt.title('Normalized Difference Vegetation Index (NDVI)')
plt.show()

# Clean up temporary files
for temp_file in [red_path, nir_path]:
    if os.path.exists(temp_file):
        os.remove(temp_file)

## 5. Converting Between Raster and xarray Datasets

xarray is a powerful library for working with labeled multi-dimensional arrays. Let's convert our raster data to an xarray Dataset:

In [ ]:
# Import the function from our package
from geoprocessor.raster.processing import to_xarray

# Convert the raster to an xarray Dataset
ds = to_xarray(sample_raster_path)

# Explore the dataset
print("xarray Dataset information:")
print(ds)

# Visualize using xarray's plotting capabilities
plt.figure(figsize=(12, 8))
ds.data.sel(band='band_1').plot(cmap='viridis')
plt.title('Visualization with xarray')
plt.tight_layout()
plt.show()

# Perform some basic analysis using xarray
print("\nBasic statistics:")
print(f"Mean value: {ds.data.mean().values}")
print(f"Min value: {ds.data.min().values}")
print(f"Max value: {ds.data.max().values}")

## 6. Performing Raster Calculations

Let's use our raster calculator function to perform simple map algebra:

In [ ]:
# Import the function from our package
from geoprocessor.raster.processing import raster_calculator

# For demonstration, we'll create a couple of synthetic rasters
with rasterio.open(sample_raster_path) as src:
    data = src.read(1)
    profile = src.profile.copy()
    
    # Create two synthetic rasters
    raster1_path = 'temp_raster1.tif'
    raster2_path = 'temp_raster2.tif'
    
    # Raster 1: Elevation (synthetic)
    elevation = data.astype(np.float32) * 10  # Simulated elevation
    profile.update(dtype=rasterio.float32, count=1)
    with rasterio.open(raster1_path, 'w', **profile) as dst:
        dst.write(elevation, 1)
    
    # Raster 2: Slope (synthetic)
    slope = data.astype(np.float32) / 50  # Simulated slope
    with rasterio.open(raster2_path, 'w', **profile) as dst:
        dst.write(slope, 1)

# Perform raster calculation: calculate areas where elevation > 1000 AND slope < 10
calc_result_path = 'calculation_result.tif'
expression = "np.logical_and(r1 > 1000, r2 < 10).astype(np.float32)"
raster_calculator([raster1_path, raster2_path], calc_result_path, expression)

# Visualize the result
plt.figure(figsize=(12, 8))
with rasterio.open(calc_result_path) as src:
    result_data = src.read(1)
    show(result_data, cmap='viridis', title='Raster Calculation Result')
plt.title('Areas with Elevation > 1000 and Slope < 10')
plt.tight_layout()
plt.show()

# Clean up temporary files
for temp_file in [raster1_path, raster2_path]:
    if os.path.exists(temp_file):
        os.remove(temp_file)

## 7. AWS Integration for Raster Processing

In a real-world scenario, you might store raster data in AWS S3 buckets. Let's see how to download and process raster data from S3:

In [ ]:
# Import necessary libraries for AWS
import boto3
from botocore import UNSIGNED
from botocore.client import Config

def process_raster_from_s3(bucket_name, key, local_path):
    """Download a raster from S3 and process it"""
    # Create an S3 client - unsigned for public buckets
    s3_client = boto3.client('s3', config=Config(signature_version=UNSIGNED))
    
    # Download the file
    print(f"Downloading {key} from {bucket_name}...")
    s3_client.download_file(bucket_name, key, local_path)
    print(f"Downloaded to {local_path}")
    
    # Process the raster
    visualize_raster(local_path)
    return local_path

# For this example, we'll use a publicly available Landsat 8 sample from AWS
# Note: In a real application, you would use your own AWS credentials and bucket
try:
    bucket_name = 'landsat-pds'
    key = 'c1/L8/042/034/LC08_L1TP_042034_20170616_20170629_01_T1/LC08_L1TP_042034_20170616_20170629_01_T1_B4.TIF'
    local_path = 'aws_landsat_red.tif'
    
    # Process the raster from S3
    # Note: This will only work if you have AWS credentials configured
    # or if using a public bucket with anonymous access
    process_raster_from_s3(bucket_name, key, local_path)
except Exception as e:
    print(f"AWS access error: {str(e)}")
    print("To use this feature, you need to have AWS credentials configured.")
    print("For this tutorial, we'll continue with the sample data.")

## 8. PostGIS Integration (Optional)

If you have PostGIS available, you can also store and query geospatial data in a PostgreSQL database:

In [ ]:
# Import necessary libraries for PostGIS integration
try:
    import psycopg2
    from psycopg2 import sql
    POSTGIS_AVAILABLE = True
except ImportError:
    POSTGIS_AVAILABLE = False

if POSTGIS_AVAILABLE:
    # Replace with your own PostgreSQL connection details
    def connect_to_postgis():
        """Connect to a PostGIS database"""
        try:
            conn = psycopg2.connect(
                host="localhost",
                database="geospatial",
                user="postgres",
                password="password"
            )
            print("Connected to PostGIS database!")
            return conn
        except Exception as e:
            print(f"PostGIS connection error: {str(e)}")
            return None
    
    # Try to connect to PostGIS
    conn = connect_to_postgis()
    if conn:
        # Example: Create a table for storing raster metadata
        with conn.cursor() as cur:
            cur.execute("""
                CREATE TABLE IF NOT EXISTS raster_metadata (
                    id SERIAL PRIMARY KEY,
                    name VARCHAR(255),
                    description TEXT,
                    acquisition_date DATE,
                    extent GEOMETRY(POLYGON, 4326),
                    resolution NUMERIC,
                    file_path VARCHAR(255)
                )
            """)
            
            # Example: Insert metadata for our sample raster
            with rasterio.open(sample_raster_path) as src:
                # Get the bounding box as a WKT string
                minx, miny, maxx, maxy = src.bounds
                bbox_wkt = f"POLYGON(({minx} {miny}, {maxx} {miny}, {maxx} {maxy}, {minx} {maxy}, {minx} {miny}))"
                
                # Insert the metadata
                cur.execute("""
                    INSERT INTO raster_metadata (name, description, acquisition_date, extent, resolution, file_path)
                    VALUES (%s, %s, %s, ST_GeomFromText(%s, 4326), %s, %s)
                """, (
                    "Sample Raster",
                    "A sample raster for demonstration purposes",
                    "2023-01-01",  # Example date
                    bbox_wkt,
                    src.res[0],  # Resolution
                    sample_raster_path
                ))
            
            # Commit the changes
            conn.commit()
            print("Raster metadata stored in PostGIS database!")
        
        # Close the connection
        conn.close()
else:
    print("PostGIS integration requires psycopg2 library. Install with 'pip install psycopg2-binary'.")

## Conclusion

In this notebook, we've covered several key techniques for processing raster data using Python's geospatial libraries:

1. Loading and visualizing raster data with Rasterio
2. Reprojecting rasters to different coordinate systems
3. Clipping rasters to vector geometries
4. Calculating vegetation indices (NDVI)
5. Converting between raster formats and xarray datasets
6. Performing raster calculations
7. Working with raster data stored in AWS S3
8. Integrating with PostGIS for spatial data management

These techniques form the foundation of a robust geospatial data processing workflow. The modular functions we've developed can be combined and extended to solve complex geospatial problems.

## Next Steps

To further explore geospatial data processing, you can:

1. Try the other notebooks in this series
2. Work with your own raster datasets
3. Explore the full GeoProcessor API
4. Experiment with more complex raster operations like segmentation, classification, and time-series analysis